Importing all the libraries and Data that will be neccessary for the analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

file_path = r'C:\Users\SthembisoM\Desktop\Karl\Driver score card\Trackmatic details.csv'

df = pd.read_csv(file_path)

Data Exploration: Check missing values and duplicates on the dataframe

In [2]:
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    print("Duplicate rows found:")
    print(duplicate_rows)
else:
    print("No duplicate rows found in the dataset.")

missing_values = df.isna().sum()
print("Missing values in each column:")
print(missing_values)

No duplicate rows found in the dataset.
Missing values in each column:
Route Channel                         1
Customer Channel                      3
Depot Code                            3
Depot                                 3
Route No.                             3
Route Status                          3
Shipment                          33446
Cases                                 2
Weight                                2
Mass                                  2
Driver                                3
Fleet Number                        298
Registration                          3
Customer Code                         3
Customer                              4
Major Group                           4
Major Chain                           4
Requested Delivery Date               3
Planned Route Start                   3
Planned Route End                     3
Actual Route Start                    3
Actual Route End                      3
Actual Route Duration                 3
Planned A

Drop missing data because the trackmatic detail dataset doesnt have any shipment numbers.

In [3]:
df = df.drop(columns=['Shipment'])
df = df.dropna(subset=['Driver', 'Customer'])
missing_values = df.isna().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
Route Channel                       0
Customer Channel                    0
Depot Code                          0
Depot                               0
Route No.                           0
Route Status                        0
Cases                               0
Weight                              0
Mass                                0
Driver                              0
Fleet Number                      295
Registration                        0
Customer Code                       0
Customer                            0
Major Group                         0
Major Chain                         0
Requested Delivery Date             0
Planned Route Start                 0
Planned Route End                   0
Actual Route Start                  0
Actual Route End                    0
Actual Route Duration               0
Planned Arrival                     0
Arrival Strike Flag                 0
Arrival Status                      0
Actual Arrival     

Correct datatypes

In [4]:

# List of columns with timestamps to convert
timestamp_columns = ['Requested Delivery Date', 'Planned Route Start', 'Planned Route End', 'Actual Route Start', 'Actual Route End', 'Actual Arrival', 'Planned Departure', 'Actual Departure', 'Planned MST', 'Actual TAT', 'Planned Arrival', 'Actual Route Duration']

# Loop through each timestamp column and convert its data type
for column in timestamp_columns:
    df[column] = pd.to_datetime(df[column], errors='coerce')

# Optional: Check the data types of the columns after conversion
print(df.dtypes)

Route Channel                             object
Customer Channel                          object
Depot Code                               float64
Depot                                     object
Route No.                                 object
Route Status                              object
Cases                                    float64
Weight                                   float64
Mass                                     float64
Driver                                    object
Fleet Number                              object
Registration                              object
Customer Code                            float64
Customer                                  object
Major Group                               object
Major Chain                               object
Requested Delivery Date           datetime64[ns]
Planned Route Start               datetime64[ns]
Planned Route End                 datetime64[ns]
Actual Route Start                datetime64[ns]
Actual Route End    

In [5]:

unique_route_statuses = df['Route Status'].unique()

# Print the unique strings
for status in unique_route_statuses:
    print(status)

Closed
Active
Requested


In [ ]:
df = df[~df['Route Status'].isin(['Active', 'Requested'])]
unique_route_statuses = df['Route Status'].unique()

# Print the unique strings
for status in unique_route_statuses:
    print(status)

Drop All Pull Forward or rolled over routes

In [ ]:
condition = df['Actual Route Start'].dt.date != df['Actual Route End'].dt.date

# Use the condition to exclude rows where Planned and Actual Start Dates are different
df = df[~condition]

Drop All sleepouts

In [ ]:
condition = df['Planned Route Start'].dt.date != df['Planned Route End'].dt.date

df = df[~condition]

condition = (df['Planned Route Start'].dt.date == df['Planned Route End'].dt.date)

# Use the condition to exclude rows where Planned Start Date and Planned End Date are different
df = df[condition]

In [ ]:
df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]
df['Departure Time Variance'] = df['Actual Route Start'] - df['Planned Route Start']

df['Departure Time Variance (minutes)'] = df['Departure Time Variance'].dt.total_seconds() / 60
print(df['Departure Time Variance (minutes)'])

In [ ]:
columns_to_drop = [
    'Adj Actual Wait TAT',
    'Adj Actual Off-Load TAT',
    'Adj Actual Doc TAT',
    'Off-Loading TAT per Case (sec)',
    'CSD Planned after 6pm',
    'CSD Arrived after 6pm',
    'PSD Planned after 4pm',
    'PSD Arrived after 4pm',
    'Mall Split',
    'Outlier',
    'Exception']
df = df.drop(columns=columns_to_drop)

In [ ]:

condition = (df['Actual Route Duration'].dt.hour * 60 + df['Actual Route Duration'].dt.minute) >= 60

# Use the condition to exclude rows with a Trip Duration less than 1 hour in the original DataFrame
df = df[condition]

In [ ]:
condition = ~((df['Planned Route Start'].dt.hour >= 7) | (df['Planned Route Start'].dt.hour < 1))

# Use the condition to exclude rows with Actual Start Time between 13:00 and 01:00
df = df[condition]

In [ ]:
condition = ~((df['Actual Route Start'].dt.hour >= 10) | (df['Actual Route Start'].dt.hour < 1))

#Use the condition to exclude rows with Actual Start Time between 13:00 and 01:00
df = df[condition]

In [ ]:
# Find the index of the row with the maximum 'Departure Time Variance (minutes)'
max_idx = df['Departure Time Variance (minutes)'].idxmax()

# Find the index of the row with the minimum 'Departure Time Variance (minutes)'
min_idx = df['Departure Time Variance (minutes)'].idxmin()

# Access the rows with the maximum and minimum values
max_row = df.loc[max_idx]
min_row = df.loc[min_idx]

# Print the rows
print("Row with Maximum Variance:")
print(max_row)

print("\nRow with Minimum Variance:")
print(min_row)


max_variance = df['Departure Time Variance (minutes)'].max()
min_variance = df['Departure Time Variance (minutes)'].min()

print(f"Maximum Variance: {max_variance} minutes")
print(f"Minimum Variance: {min_variance} minutes")

In [ ]:
variance_stats = df['Departure Time Variance (minutes)'].describe()
print(variance_stats)

In [ ]:
df = df['Departure Time Variance (minutes)']

# Create a histogram with specified number of bins (you can adjust this)
plt.hist(df, bins=20, edgecolor='k')

# Add labels and a title
plt.xlabel('Departure Time Variance (minutes)')
plt.ylabel('Frequency')
plt.title('Histogram of Departure Time Variance')

# Show the histogram
plt.show()

Based on these statistics:

The mean departure time variance being positive (8.38 minutes) suggests that, on average, departures tend to be slightly later than planned.

The standard deviation being relatively large (47.49 minutes) indicates a wide spread of departure time variances, with some instances of significant delays and early departures.

The presence of negative values (e.g., minimum of -255.63 minutes) indicates instances where departures occurred earlier than planned.

The maximum departure time variance of 478.35 minutes suggests that there are cases of substantial delays in departures.

In conclucsion I am satisfied with the data quality.

Creating Category for dispatching pattern

In [ ]:
# Convert 'Planned Route Start' and 'Actual Route Start' columns to datetime format
df['Planned Route Start'] = pd.to_datetime(df['Planned Route Start'])
df['Actual Route Start'] = pd.to_datetime(df['Actual Route Start'])

# Filter the DataFrame based on date comparison
df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]

# Continue with the calculations and categorizations

# Filter the DataFrame based on date comparison
#df = df[df['Planned Route Start'].dt.date == df['Actual Route Start'].dt.date]

# Calculate 'Departure Time Variance' and 'Departure Time Variance (minutes)'
df['Departure Time Variance'] = df['Actual Route Start'] - df['Planned Route Start']
df['Departure Time Variance (minutes)'] = df['Departure Time Variance'].dt.total_seconds() / 60

# Define the categorization function
def categorize_departure_variance(minutes):
    if minutes <= -120:
        return 'Dispatched 2hrs+ Earlier'
    elif minutes > -120 and minutes <= -60:
        return 'Dispatched 1-2 hrs early'
    elif minutes > -60 and minutes <= -30:
        return 'Dispatched 30min-1hr early'
    elif minutes > -30 and minutes <= 30:
        return 'On Time'
    elif minutes > 30 and minutes <= 60:
        return 'Dispatched 30min-1hr late'
    elif minutes > 60 and minutes < 120:
        return 'Dispatched 1hr-2hrs late'
    else:
        return 'Dispatched 2hrs+ late'

# Apply the custom function to create a new column
df['Dispatch Variance Category'] = df['Departure Time Variance (minutes)'].apply(categorize_departure_variance)

# Display the DataFrame with the new category column
print(df[['Departure Time Variance (minutes)', 'Dispatch Variance Category']])


In [ ]:
# Convert 'Planned Route End' and 'Actual Route End' columns to datetime format
df['Planned Route End'] = pd.to_datetime(df['Planned Route End'])
df['Actual Route End'] = pd.to_datetime(df['Actual Route End'])

# Filter the DataFrame based on date comparison
df = df[df['Planned Route End'].dt.date == df['Actual Route End'].dt.date]

# Calculate 'Depot Return Variance' and 'Depot Return Variance (minutes)'
df['Depot Return Variance'] = df['Actual Route End'] - df['Planned Route End']
df['Depot Return Variance (minutes)'] = df['Depot Return Variance'].dt.total_seconds() / 60

# Define the categorization function
def categorize_departure_variance(minutes):
    if minutes <= -120:
        return 'Returned 2hrs+ Earlier'
    elif minutes > -120 and minutes <= -60:
        return 'Returned 1-2 hrs early'
    elif minutes > -60 and minutes <= -30:
        return 'Returned 30min-1hr early'
    elif minutes > -30 and minutes <= 30:
        return 'On Time'
    elif minutes > 30 and minutes <= 60:
        return 'Returned 30min-1hr late'
    elif minutes > 60 and minutes < 120:
        return 'Returned 1hr-2hrs late'
    else:
        return 'Returned 2hrs+ late'

# Apply the custom function to create a new column
df['Depot Return Variance Category'] = df['Depot Return Variance (minutes)'].apply(categorize_departure_variance)

# Display the DataFrame with the new category column
print(df[['Depot Return Variance (minutes)', 'Depot Return Variance Category']])


In [ ]:
# Convert 'Actual Route End' and 'Actual Route Start' columns to datetime format
df['Actual Route End'] = pd.to_datetime(df['Actual Route End'])
df['Actual Route Start'] = pd.to_datetime(df['Actual Route Start'])

# Convert 'Actual TAT' and 'Planned MST' columns to datetime
df['Actual TAT'] = pd.to_datetime(df['Actual TAT'], errors='coerce')
df['Planned MST'] = pd.to_datetime(df['Planned MST'], errors='coerce')

# Filter the DataFrame to include only rows where 'Actual Route End' and 'Actual Route Start' have the same date
df = df[df['Actual Route End'].dt.date == df['Actual Route Start'].dt.date]

# Calculate 'TAT Variance' and 'TAT Variance (Minutes)'
df['TAT Variance'] = df['Actual TAT'] - df['Planned MST']
df['TAT Variance (Minutes)'] = df['TAT Variance'].dt.total_seconds() / 60

# Filter the DataFrame to keep only rows with TAT Variance greater than or equal to 20 minutes
df = df[df['TAT Variance (Minutes)'] >= 20]

# Reset the index of the DataFrame
df.reset_index(drop=True, inplace=True)

def categorize_TA_variance(minutes):
    if minutes > 0 and minutes <= 30:
        return '0 <TAT <=30'
    elif minutes > 30 and minutes <= 60:
        return '30 <TAT <=60'
    elif minutes > 60 and minutes <= 120:
        return '60 <TAT <=120'
    elif minutes > 120 and minutes <= 180:
        return '120 <TAT <=180'
    elif minutes > 180 and minutes <= 240:
        return '180 <TAT <=240'
    else:
        return 'TAT 4hrs +'

# Apply the custom function to create a new column
df['TAT Variance Category'] = df['TAT Variance (Minutes)'].apply(categorize_departure_variance)

# Calculate the total count for each driver
driver_counts = df.groupby('Driver').size().reset_index(name='Total')

# Pivot the DataFrame to get dispatch indicator counts
pivot_df = df.pivot_table(index='Driver', columns='Depot Return Variance Category', aggfunc='size', fill_value=0)

# Calculate percentages for each dispatch indicator
percentage_df = pivot_df.div(driver_counts['Total'], axis=0) * 100

# Reset the index for the percentage DataFrame
percentage_df.reset_index(inplace=True)

# Rename columns for clarity
percentage_df.columns.name = None

# Display the resulting DataFrame
print(percentage_df)

category_counts = df.groupby(['Driver', 'TAT Variance Category']).size().reset_index(name='Count')
print(category_counts)
# Display the DataFrame with the new category column
print(df[['TAT Variance (Minutes)', 'TAT Variance Category']])






In [ ]:
variance_stats = df['TAT Variance (Minutes)'].describe()
print(variance_stats)

In [ ]:
df = df[['TAT Variance (Minutes)']]

# Create a histogram with specified number of bins (you can adjust this)
plt.hist(df, bins=20, edgecolor='k')

# Add labels and a title
plt.xlabel(['TAT Variance (Minutes)'])
plt.ylabel('Frequency')
plt.title('Histogram of TAT Variance')

# Show the histogram
plt.show()